# Image Analysis for Geospatial Application

## Lab 2: Logistic Regression classifier & Pytorch basics

__Enter your data:__

Please enter your names below (double click to edit the table).

| Name  | Matr.-nr. |
|-|-|
| your name | 12345 |

`In this lab, you will learn how to implement Logistic Regression classifiers using both the Scikit-learn and Pytorch frameworks and how to apply them to randomly generated data samples and existing benchmark datasets for classification purposes.`

__Note__: Please read the instructions for each exercise carefully before answering. You __should not use any external libraries except those that are implemented in the code cell below__!

Required __libraries__ for this lab can be installed by:

__1__. running the following command in a code cell ``!conda install numpy matplotlib scikit-learn`` if not installed already, and 

__2__. visiting and following the instructions at [pytorch.org](https://pytorch.org/get-started/locally/) to see how __Pytorch__ and __Torchvision__ modules can be installed. The CUDA tab refers to the support of GPUs. Unless you have experience with the CUDA framework select `None` here.

Required __imports__ and __settings__ for this lab:

In [ ]:
# IMPORTS
import lab                                      # Given functions
import numpy as np                              # Numerical computations
from sklearn.metrics import confusion_matrix
import matplotlib                               # Plots
import matplotlib.pyplot as plt
import torch                                    # PyTorch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

# GLOBAL SETTINGS
PlotSize = 7                                    # Size of plots
matplotlib.rcParams['figure.figsize'] = [PlotSize, PlotSize] 
CMAP = plt.cm.Accent                            # Color mapping 
np.set_printoptions(precision=3)                # Array print precision
np.random.seed(0)

# create the Data folder
lab.create_data_folder()

## Exercise 1: Discriminative Probabilistic Classifiers -  (25%)

In this exercise, we will use the toy dataset we generated in the first lab. 

Use the following link [to download the data file](https://seafile.cloud.uni-hannover.de/f/a13a894abe27425ab785/). The file `toy_data.pickle` __should be placed in the `Data` folder__ where this notebook is run. 

__Run__ the next cell to load the data. 

If you encounter problems with *pickle* package while loading the file, check the version of *numpy*. You may need to install *numpy==1.24.3* with the command: ``!conda install numpy==1.24.3``. This is because the file was generated by the indicated numpy version.

In [ ]:
import pickle
    
# LOAD TOY DATASET
with open('./Data/toy_data.pickle', 'rb') as file:
    Xtoy, ytoy, Xtoy_t, ytoy_t = pickle.load(file)
    
# Define the training set features, training set labels,  test set features, test set labels
X, y, X_t, y_t = Xtoy, ytoy, Xtoy_t, ytoy_t

# Check whether the data loading is successfully completed:

print(f'Training set features: {X.shape}')
print(f'Training set labels: {y.shape}')
print(f'Testing set features: {X_t.shape}')
print(f'Testing set labels: {y_t.shape}')

### Exercise 1.1: Logistic regression -  10%

In the next cell, starter code for a class implementing Logistic Regression is given. The class design is again adapted from the scikit-learn module. 

__Complete__ the method ``fit(...)`` that iteratively determines the weights of the classifier.

__Note__ that the weights are internally stored in a 2D array (for faster inference). In the missing part of the method you can treat $\nabla E$ as column vector (like in the lecture). The reshaping is already implemented.

In [ ]:
class LogReg():
    def __init__(self, num_classes, num_features, sigma=None):
        # Initializes the classifier
        #  num_classes: number of classes
        #  num_features: number of features (length of each feature vector)
        #  sigma: regularization term or None for training without regularization
        
        self.num_classes = M = num_classes
        self.num_features_w_bias = F = num_features + 1  # Internaly the bias is handled as additional feature   
        self.weights = np.random.uniform(size=(M, F))    # Init. weights using random normally distributed values
        self.sigma = sigma
    
    def fit(self, X, y, max_iter=100, crit=0.0001):
        # Computes gradient and Hesse matrix for each class using ML
        # update the weights using Newton-Raphson method
        #  X: feature_vectors [num_feature_vectors x num_features]
        #  y: corresponding labels [num_feature_vectors]
        #  max_iter: maximum number of iterations
        #  crit: stopping criterion L2 of nabla_E
        
        X = np.copy(X) # to prevent errors due to modifications of these arrays
        y = np.copy(y)
        
        X = np.hstack((np.ones((X.shape[0],1)), X))      # Add bias feature 
        N, F = X.shape                                   # N: num_samples
        M = self.num_classes                             # M: num_classes

        for it in range(max_iter):
            nabla_E = np.zeros(((M-1)*F, 1))             # Init. nabla_E
            H = np.zeros(((M-1)*F, (M-1)*F))             # Init. H
            
            ##########################################################
            
            # YOUR CODE GOES HERE!
            #
            # 1. Get predictions Y
            # 2. Define the Binary indicator as well as the unit matrix I[j,k]
            # 3. Update nabla_E and H
            # 4. Add regularization if sigma is not None

            if not self.sigma is None:
                # Add regularization terms to H and nabla_E 

            ###########################################################
            
            H_inv = np.linalg.inv(H)                     # Update of weights ..
            update = np.dot(H_inv, nabla_E)                 #
            self.weights[1:] -= update.reshape(((M-1), F)) 
            
            max2nab = np.sqrt(np.sum(np.square(nabla_E)))   # Check convergence
            if max2nab < crit:
                break
                
        print('weights:\n', self.weights)
            

    def compute_posteriors(self, X):
        # Computes posteriors for feature vectors
        #  X: feature_vectors [num_feature_vectors x num_features]
        
        num_samples = X.shape[0]                                  # Number of samples to predict
        posteriors = np.zeros((num_samples, self.num_classes))    # Init. posterior matrix
        
        if X.shape[1] < self.num_features_w_bias:                 # Add bias feature (if not done already)
            X = np.hstack((np.ones((X.shape[0],1)), X))
            
        for xi, x in enumerate(X):
            a = np.dot(self.weights, x)                           # Numerical trick for stability
            a -= np.max(a)
            eWx = np.exp(a)
            posteriors[xi] = eWx/np.sum(eWx)
            
        return posteriors

    def predict(self, X):
        # Predicts labels for feature vectors in X
        #  X: feature_vectors [num_feature_vectors x num_features]
        
        P = self.compute_posteriors(X)
        return np.argmax(P, axis=1)

In the next cell a classifier is created and fitted to the training data. __Run the cell__ to check your implementation.

In [ ]:
logreg = LogReg(num_classes=4, num_features=2, sigma=5)

# Scaling of features for numerical reasons! 
scale = 1 / np.max(X)

# Train the classifier
_ = logreg.fit(X * scale, y, max_iter=20)

# Create a set of 'all' features in the limits
xx, yy = np.meshgrid(np.arange(0, 256, 1), np.arange(0, 256, 1))
mesh_features = np.c_[xx.ravel(), yy.ravel()]

# Get posteriors for the new test samples
PROB = logreg.compute_posteriors(mesh_features * scale)
lab.print_probabilities(PROB, (256, 256), 'Posterior', n_cls=4)

Again, we can plot the decision boundaries:

In [ ]:
C = logreg.predict(mesh_features * scale)
lab.print_decision_boundaries(C, (256, 256))

### Exercise 1.2: Logistic regression with feature space transformation -  5%

__Implement__ the function ``transform_features`` in the cell below. It takes $X$, a $N\times2$ feature array, and returns $X2$, the transformed $N\times5$ feature array. Each row in $X2$ should contain the original features $(x, y)$ as well as the squares $(x^2, y^2)$ and the mixed product $(x\cdot y)$. For numerical reasons, a scaling of features (scale) needs to be applied to features before transformation.

In [ ]:
def transform_features(X, scale):
    
    # YOUR CODE GOES HERE!
    
    X2 =
    
    return X2

__Run__ the following cell to transform the features of the training and test sets.

In [ ]:
X2   = transform_features(X, scale)
X2_t = transform_features(X_t, scale)

The next cell will train a second classifier based on the transformed features.

In [ ]:
logreg2 = LogReg(num_classes=4, num_features=5, sigma=5)
logreg2.fit(X2, y)

# Get posteriors for the new test samples
PROB = logreg2.compute_posteriors(transform_features(mesh_features, scale))
lab.print_probabilities(PROB, xx.shape, 'Posterior', n_cls=4)

Again, we can plot the decision boundaries:

In [ ]:
C = logreg2.predict(transform_features(mesh_features, scale))
lab.print_decision_boundaries(C, (256, 256))

### Exercise 1.3: Comparison and evaluation -  10%

Use your implementation of the function ``compute_quality_metrics``  from the previous Lab in the cell below.

Note that it should computes the following quality metrics (all in the range between 0 and 1):

- Precision per class (1D array)
- Recall per class (1D array)
- F1-score per class (1D array)
- Overall accuracy (scalar)
- Mean F1-score  (scalar)

The function takes the array of predictions $Y$, the corresponding reference labels $y$ and the number of classes $C$ as input.

In [ ]:
def compute_quality_metrics(Y, y, C):

    # YOUR CODE GOES HERE!
    
    precisions = 
    recalls = 
    f1_scores = 
    overall_accuracy = 
    mean_f1_score = 
    
    return precisions, recalls, f1_scores, overall_accuracy, mean_f1_score

__Run__ the next cell to test your implementation. The code computes and prints the overall accuracy for both datasets and both classifiers.

In [ ]:
_, _, _, oa_lr_train, mf1_lr_train = compute_quality_metrics(logreg.predict(X * scale), y, 4)
_, _, _, oa_lr_test, mf1_lr_test = compute_quality_metrics(logreg.predict(X_t * scale), y_t, 4)

_, _, _, oa_lr2_train, mf1_lr2_train = compute_quality_metrics(logreg2.predict(X2), y, 4)
_, _, _, oa_lr2_test, mf1_lr2_test = compute_quality_metrics(logreg2.predict(X2_t), y_t, 4)

print('Overall Accur. | TRAIN-SET| TEST-SET\n' + '-' * 37)
print('LOG. REG.       |  {:.2%}  |  {:.2%}'.format(oa_lr_train, oa_lr_test))
print('LOG. REG. W/ TR.|  {:.2%}  |  {:.2%}'.format(oa_lr2_train, oa_lr2_test))
print('\n')
print(' Mean F1-Score  | TRAIN-SET| TEST-SET\n' + '-' * 37)
print('LOG. REG.       |  {:.2%}  |  {:.2%}'.format(mf1_lr_train, mf1_lr_test))
print('LOG. REG. W/ TR.|  {:.2%}  |  {:.2%}'.format(mf1_lr2_train, mf1_lr2_test))

__Write__ a brief discussion, which answers the following questions:

- Write a paragraph describing the main steps to train a logistic regression model? This should be answered w.r.t. the methodology, not the implementation.

- What is the idea behind the regularization and how does the choice of $\sigma$ affect the results? Try different values for $\sigma$,  document and discuss the results. 

- Explain the idea of the quadratic expansion and the influence on the quality metrics. Would a cubic expansion make sense here?

#### Discussion:

*Write the discussion here. Do not forget to answer all questions, item by item, and to identify which answer belongs to which question.*

## Exercise 2: PyTorch basics -  (35%)

In this part you should get comfortable with the `pytorch` framework.

#### Binary toy dataset

We start by creating a toy data set for a binary classification problem. Each sample $\mathbf{x}_i$ consists of two features $x_0$ and $x_1$.

In [ ]:
num_samples = 600

# Samples for class 0
samples_C0 = np.vstack((np.random.randn(num_samples//6, 2) * 0.075 + np.array([[0.75, 0.15]]),  # Samples for class 0
                       np.random.randn(num_samples//6, 2) * 0.1 + np.array([[0.7, 0.72]]),
                       np.random.randn(num_samples//6, 2) * 0.075 + np.array([[0.25, 0.75]])))
# Samples for class 1
samples_C1 = np.random.randn(num_samples//2, 2) * 0.15 + np.array([[0.35, 0.4]])                # Samples for class 1

all_samples = np.concatenate((samples_C0, samples_C1))                                          # Concat the samples
reference_values = np.concatenate((np.zeros(num_samples//2),                                    # Target labels
                                   np.ones(num_samples//2))).astype(dtype=np.float32)

matplotlib.rcParams['figure.figsize'] = [PlotSize, PlotSize]                                    # Plot samples 
plt.plot(samples_C0[:,0], samples_C0[:,1], 'b.')
plt.plot(samples_C1[:,0], samples_C1[:,1], 'g.')
plt.xlim(0, 1); plt.ylim(0, 1); plt.show()

### Logistic regression with gradient descent
Because the logistic regression model is relatively simple and very close to the concept of neural networks, we start by implementing a logistic regression using the `pytorch` framework. The first step is to convert input samples and reference labels to pytorch tensors, here `X` and`Y`, respectively. The features of torch are mostly comparable to those of numpy. A `torch.tensor` is the fundamental class of the framework, comparable to the `numpy.ndarray` in numpy.

In [ ]:
X = torch.tensor(all_samples, dtype=torch.float)  # A tensor can be created from a ndarray, here with explicid datatype
Y = torch.tensor(reference_values).float()        # Casting the datatype is also possible with the according functions
print(X.shape, Y.shape)      

Next, we will set up the trainable parameters of the model. Remember that the probability $P$ of a feature $x$ for belonging to class 1 is defined as

$$P(x = (x_0, x_1)~~|~~C=1) = \sigma(w_0 \cdot x_0 + w_1 \cdot x_1 + b)$$

for a binary classification problem. Here, $\sigma$ is the sigmoid function:

$$\sigma(x) = \dfrac{1}{1+e^{-x}}$$

Three trainable parameters are required here:

In [ ]:
w_0 = torch.tensor([0.0], requires_grad=True)     # All parameters are initialized as a scalar with zero value
w_1 = torch.tensor([0.0], requires_grad=True)     # 'requires_grad' enables automatic differentiation
b   = torch.tensor([0.0], requires_grad=True)
params = [w_0, w_1, b]

The next cell performs one update step. __Run the cell multiple times__, to see how the parameters change and the cross entropy loss shrinks.

In [ ]:
print('Values of w0/w1/b:     {:.4f} / {:.4f} / {:.4f}'.format(        # Print the current values of the parameters.
    float(w_0.data), float(w_1.data), float(b.data)))                  # The values can be accessed by the 'data' field

P = torch.sigmoid(w_0*X[:,0] + w_1*X[:,1] + b)                         # Compute the output for all samples  
P = P.view(-1)                                                         #   and flatten to a 1D tensor

E = -1 * torch.mean((Y)*torch.log(P) + (1 - Y)*torch.log(1 - P))       # Computation of the CE loss
print('Value of loss E:       {:.5f}'.format(float(E.data)))

E.backward()                        # The backward function will compute the gradients for all parameters,
                                    #  that the tensor depends on (here, E depends on w_0, w_1 and b)

print('Gradients of w0/w1/b:  {:.4f} / {:.4f} / {:.4f}'.format(        # The gradients are stored in the respective 
    float(w_0.grad.data), float(w_1.grad.data), float(b.grad.data)))   #   tensors and are accessed via 'grad.data'

LR = 0.5                            # Define a learning rate and 
for p in params:
    p.data.add_(-LR * p.grad.data)  #  update the parameters (opposite direction of the gradients)
    p.grad.data.zero_()             #  each call of 'backward' would accumulate the gradients so we set them to zero

### Exercise 2.1: Training loop -  10%

__Complete__ the function `train()` in which the logistic regression model is trained for `num_iter` iterations with gradient descent and a learning rate of `LR`. After training has finished, the function should compute and return the overall accuracy of the last predictions `OA`, a list of all losses `Es` and the final values of the parameters.

In [ ]:
def train(LR, num_iter):
    Es = []                                        # Append cross entropy loss of each iteration to this list
    w_0 = torch.tensor([0.0], requires_grad=True)  # Overwrite / initialize the parameters (hard reset)
    w_1 = torch.tensor([0.0], requires_grad=True)
    b   = torch.tensor([0.0], requires_grad=True)
    params = [w_0, w_1, b]
    
    # YOUR CODE GOES HERE
    
    f_params = [float(p.data) for p in params]     # Store final parameters as float values
    return Es, OA, f_params

In the next cell, the classifier is trained with different learning rates. The plot will show the loss over training iterations. Use this as a check for your implementation.

In [ ]:
matplotlib.rcParams['figure.figsize'] = [PlotSize*2, PlotSize//2]  

all_f_params = []
LRs = [0.5, 0.1, 0.01]
for LR in LRs:
    Es, OA, f_params = train(LR, 2500)
    all_f_params.append(f_params)
    print('LR: {}, OA: {:.3%}'.format(LR, OA))
    plt.plot(Es, label='LR: {}'.format(LR))
    plt.title('Training loss')

plt.ylabel("CE Loss"); plt.xlabel("Iteration")
plt.legend(); plt.ylim(0, 1); plt.show()

__Visualize__ the decision boundaries:

In [ ]:
matplotlib.rcParams['figure.figsize'] = [PlotSize, PlotSize]  

plt.plot(samples_C0[:, 0], samples_C0[:, 1], 'b.')
plt.plot(samples_C1[:, 0], samples_C1[:, 1], 'g.')

x0, x1 = 0.0, 1.0

for i, ((w_0i, w_1i, b_i), LR) in enumerate(zip(all_f_params, LRs)):    
    y0 = (-x0 * w_0i - b_i) / w_1i
    y1 = (-x1 * w_0i - b_i) / w_1i
    plt.plot([x0, x1], [y0, y1], label= 'Decision boundary (LR: {})'.format(LR))
plt.legend(); plt.xlim(0, 1); plt.ylim(0, 1); plt.show()

### Exercise 2.2: Neural networks -  15%

Before we start with neural networks, please  read the next two cells __carefully!__ Here, the class `torch.nn.Module` is introduced. It is used to model a network architecture. In the next cell the logistic regression is implemented as such a module:

In [ ]:
class LogReg(nn.Module):                # A module is created as a class which is derived from torch.nn.Module
    
    def __init__(self):                 # The __init__ method is implicitly called whenever a new instance is created
        super(LogReg, self).__init__()  #  - Parents' __init__ method must be called (passing the current class)
        self.fcD = nn.Linear(2, 1)      #  - Defining submodules e.g. dense or convolutional layers
                                        #    the `Linear` module creates a dense layer. 
                                        #    Arguments are number of in / out neurons and 
                                        #    whether to use a bias or not (default is true)
        
    def forward(self, x):               # The 'forward' method describes how input is mapped to the output
                                        # (It is common to reuse a single variable in sequential networks - here, x)
        x = self.fcD(x)                 # First operation: pass input(x) through dense layer
        x = torch.sigmoid(x)            # Second operation: apply non-linarity
        return x                        # Return the output

When using torch modules, the computation of the loss as well as the update steps can be simplified. This is shown in the next cell. 

In [ ]:
IT = 2500         # Hyperparameters: - Number of training iterations (Steps)
LR = 0.5          #                  - Learning rate

net = LogReg()                                  # Initialize the network
optimizer = optim.SGD(net.parameters(), lr=LR)  # Define the optimizer, here SGD: Stochastic Gradient Descent
                                                # The optimizer performs the update step and resets gradients

Ls = []                         #  We want to keep track of the loss over iterations
criterion = nn.BCELoss()        #  Loss models are used to compute loss, here: BCE Binary Cross Entropy 

for i in range(IT):             # Training loop:
    optimizer.zero_grad()       #   Reset gradients
    P = net(X)                  #   Feed forward step
    P = P.view(-1)              #   Flatten the pridictions to 1D-tensor
    loss = criterion(P, Y)      #   Feed predictions and reference to the loss model
    loss.backward()             #   Back propagation (compute gradients for all variables)
    optimizer.step()            #   Perform gradient descent
    Ls.append(float(loss.data)) #   Store loss (converted from tensor to float)

predicted_labels = np.round(P.data.numpy())                     # Compute labels by rounding (threshold = 0.5)
corrects = np.sum(predicted_labels == Y.data.numpy())           # Count correct predictions
print('Overall accuracy: {:.3%}'.format(corrects/num_samples))  # Compute and show overall accuracy

matplotlib.rcParams['figure.figsize'] = [PlotSize*2, PlotSize//2]             # Plot cross entropy
plt.ylabel("CE Loss"); plt.xlabel("Iteration")
plt.title('Training loss')
plt.plot(Ls, label='Logistic Regression'); plt.ylim(0, 1); plt.legend(); plt.show()

Run the next cell to __visualize__ the decision boundaries of the model:

In [ ]:
matplotlib.rcParams['figure.figsize'] = [0.6*PlotSize, 0.5*PlotSize]  

xx, yy = np.meshgrid(np.arange(0, 1, 1/150), np.arange(0, 1, 1/150))
grid_samples = np.c_[xx.ravel(), yy.ravel()].reshape(-1,2).astype(np.float32)

Z = net(torch.tensor(grid_samples)).data.numpy().reshape(xx.shape)

plt.pcolormesh(xx, yy, Z, cmap="seismic", shading='auto')
plt.plot(samples_C0[:, 0]*1, samples_C0[:, 1]*1, 'b.')
plt.plot(samples_C1[:, 0]*1, samples_C1[:, 1]*1, 'g.')

plt.axis("equal"); plt.colorbar(); plt.xlim(0,1); plt.ylim(0,1); plt.show()

Now __implement__ a multilayer perceptron (neural network). The architecture should have:

- A variable number of hidden layers (specified by the argument `layers`)
- A variable number of neurons in each hidden layer (specified by the argument `neurons`)
- Use the `tanh` non-linearity for the hidden layers. (E.g. by using `torch.tanh()`)
- Use the `sigmoid` non-linearity for the output layer. 

Note that for a variable number of layers you have to use either a
[ModuleList](https://pytorch.org/docs/stable/generated/torch.nn.ModuleList.html) or the [Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) module.


In [ ]:
class NN(nn.Module): 
    def __init__(self, layers, neurons):
        super(NN, self).__init__()
        
        # YOUR CODE GOES HERE
        
    def forward(self, x):
        
        # YOUR CODE GOES HERE
        return x

The training loop remains similar, but several `settings` are compared. Also training is now done using mini batches. __Add your own new settings__ (minimum 5 settings)  and observe the effect of changing the hyper-parameters of the architecture and the training.

In [ ]:
matplotlib.rcParams['figure.figsize'] = [PlotSize*2, PlotSize//2]  

#  The settings are stored as a list of 5-tuples. The parameters are:
# (iterations, learning rate, num. layers, num. neurons per layer, batch size) 

settings = [(3000, 0.5, 2, 8, 64),]  # currently, one example setting; ADD YOUR NEW ONES TO THE LIST

for si, (IT, LR, layers, neurons, batch_size) in enumerate(settings):
    print("setting {}) LR: {}, #Layers: {}, #Neurons: {}, Batch size {}".format(
        si+1, LR, layers, neurons, batch_size))
    
    Ls = []
    criterion = nn.BCELoss()
    net = NN(layers, neurons)
    optimizer = optim.SGD(net.parameters(), lr=LR)

    for i in range(IT):
        idx = torch.randperm(X.size(0))[:batch_size]
        x, y = X[idx], Y[idx]
        optimizer.zero_grad()  
        pred = net(x).view(-1)
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()  
        Ls.append(float(loss.data))

    pred = net(X).view(-1)
    predicted_labels = np.round(pred.data.numpy())
    corrects = np.sum(predicted_labels == Y.data.numpy())
    print('Overall accuracy: {:.3%}'.format(corrects/num_samples))

    plt.subplot(1,2,1)
    plt.plot(Ls); plt.ylim(0, 1); plt.ylabel("CE Loss"); plt.xlabel("Iteration")

    Z = net(torch.tensor(grid_samples)).data.numpy().reshape(xx.shape)

    plt.subplot(1,3,3)
    plt.pcolormesh(xx, yy, Z, cmap="seismic", shading='auto')
    plt.plot(samples_C0[:, 0]*1, samples_C0[:, 1]*1, 'b.')
    plt.plot(samples_C1[:, 0]*1, samples_C1[:, 1]*1, 'g.')

    plt.axis("equal"); plt.xlim(0,1); plt.ylim(0,1); plt.show()

### Exercise 2.3:  Discussion -  10%

- Write a discussion based on the settings in the previous cell. Run several settings and discuss the influence of the hyper-parameters: learning rate, number of layers, number of neurons per layer and batch size. A recommended strategy is to use one baseline setting and then add scenarios where one hyper-parameter is changed, respectively. For each hyper-parameter argue with what you observe in the experiments but also from a theoretical viewpoint. 


#### Discussion:

*Write the discussion here. Do not forget to answer all questions, item by item, and to identify which answer belongs to which question.*

## Exercise 3: Image classification -  (40%)

In the previous exercise, we have learned how to implement a perceptron (Logistic Regression) and a Multilayer Perceptron for binary classification using a toy dataset. In this exercise, we are going to apply the same models for classifying images using the __pytorch__ framework on the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset. It contains images of hand written digits. Each image has 28 x 28 pixels and one channel. The classes correspond to the digits 0-9. The given function `lab.MnistGenerator()` will download the dataset (if not already available). The images in the MNIST dataset are already split into:

- 60.000 images for training
- 10.000 images for testing

We will preserve 1.000 training images as a validation dataset.

Run the next cell to visualize few data samples from training. You can run the cell several times to see some digits:

In [ ]:
gen = lab.MnistGenerator()
Xs, Ys = gen.get_train_batch()

figure = plt.figure(figsize=(10, 8))
cols, rows = 3, 3
print('Examples of digits:')

for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(Xs), size=(1,)).item()
    img, label = Xs[sample_idx], Ys[sample_idx]
    title = "Class {}:".format(label)
    figure.add_subplot(rows, cols, i)
    plt.title(title)
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
plt.show()

### Exercise 3.1: Defining the models -  15%

We will train two different models on the dataset:

__1. Logistic regression:__

- No hidden layer, direct mapping from (normalized) input to output.

__2. Multilayer perceptron:__

- One hidden layer with 256 neurons and tanh non-linearity.

#### Hints:

- In each forward pass __normalize__ the images to a range of -1 to 1 (input values are in range 0 to 255).
- Use `x=x.view(-1, N)` to __flatten__ a feature map (with `N` elements). This is required when passing images or feature maps to dense layers.
- In all two cases __the last layer must not have a non-linearity__ (because the softmax is included in the loss formulation we will use later)

In [ ]:
class LogReg(nn.Module):
    def __init__(self):
        # ENTER YOUR CODE HERE
        
    def forward(self, x):
        # ENTER YOUR CODE HERE
        
        return x

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        # ENTER YOUR CODE HERE
        
    def forward(self, x):
        # ENTER YOUR CODE HERE
        
        return x

#### Training loop
The training loop is given in the next cell. It is similar to the previous training loop, but additionally, an evaluation on the validation is performed.

__Complete__ the function to include the evaluation on test set.

In [ ]:
def train_network(gen, net, optimizer, num_iter):
    Ls = []
    TBAs = []
    VAs = []
    
    net.train()
    criterion = nn.CrossEntropyLoss()
    
    for i in range(num_iter):
        X, Y = gen.get_train_batch()
        num_samples = float(len(Y))
        
        TX = torch.tensor(X).view(-1, 1, 28, 28)
        TY = torch.tensor(Y).long()
        
        optimizer.zero_grad()  
        pred = net(TX)
        loss = criterion(pred, TY)
        loss.backward()
        optimizer.step()  

        Vs, Is = torch.max(pred, -1)
        num_correct = np.sum(Is.data.numpy() == Y)
        TBA = num_correct/num_samples
        print('\rIt {}/{}, CE: {:.3f}, Batch Acc.: {}/{} = {:.1%}'.format(
            i, num_iter, loss.data, num_correct, len(Y), TBA), end='')
        
        # Compute accuracy of validation set
        
        X, Y = gen.get_validation_batch()
        num_samples = float(len(Y))
        TX = torch.tensor(X).view(-1, 1, 28, 28)
        TY = torch.tensor(Y).long()
        net.eval()
        pred = net(TX)
        net.train()
        Vs, Is = torch.max(pred, -1)
        num_correct = np.sum(Is.data.numpy() == Y)
        VA = num_correct/num_samples
        
        Ls.append(float(loss.data))
        TBAs.append(TBA)
        VAs.append(VA)
        
    # Compute accuracy of test set
        
    X, Y = gen.get_test_batch()
    
    # ENTER YOUR CODE HERE
    #
    # 1. Convert X and Y into Tensors
    # 2. Set the network into inference mode
    # 3. Make predictions
    
    CM = confusion_matrix(Y, Is)
    return Ls, TBAs, VAs, CM

### Exercise 3.2: Training -  10%

In this section the models are trained (with different settings).
In each cell, __choose appropriate hyperparameters__ to achieve stable results. In particular choose:

- Batch size `BS`
- Number of iterations `IT`
- Learning rate `LR`

#### Training: Logistic Regression

In [ ]:
BS = 
LR = 
IT = 

gen = lab.MnistGenerator(BS) # Data generator
net = LogReg() 

optimizer = optim.SGD(net.parameters(), lr=LR)

Ls, TBAs, VAs, CM = train_network(gen, net, optimizer, IT)
matplotlib.rcParams['figure.figsize'] = [PlotSize*2, PlotSize//2]
lab.print_summary(Ls, TBAs, VAs, CM)

Because the logistic regression has no intermediate layers, the the learned weights can be  interpreted visually. The following code will show the feature maps as images. You should be able to see patterns that correspond to the digits 0-9.

In [ ]:
Is = []
weights = net.fc1.weight.data.numpy()
for i in range(10):
    Is.append(weights[i].reshape((28,28)))
    Is.append(np.ones((28,1))*np.min(weights))
I = np.hstack(Is)
plt.imshow(I, cmap='copper')
plt.show()

#### Training: Multilayer perceptron (Neural network)

In [ ]:
BS = 
LR = 
IT =

gen = lab.MnistGenerator(BS)
net = MLP() 
optimizer = optim.SGD(net.parameters(), lr=LR)

Ls, TBAs, VAs, CM = train_network(gen, net, optimizer, IT)
lab.print_summary(Ls, TBAs, VAs, CM)

### Exercise 3.3: Discussion  -  15%

__Write a discussion__ which answers the following aspects and questions: 

- During all experiments, what was the main influence of learning rate and batch size?

- Discuss the performance of both models (LogReg and MLP). What might be the reasons behind the differences in the performance you observe?

- Describe existing techniques that can be used to improve the classification results of a MLP.

#### Discussion:

*Write the discussion here. Do not forget to answer all questions, item by item, and to identify which answer belongs to which question.*